In [34]:
# 正式作業
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
from datetime import datetime
from tkinter import messagebox
import time
import csv
import urllib
import tkinter as tk
import tkinter.ttk as tt
import pandas as pd
import numpy as np


# 定義方程式
# 定義click方程式
def choose(option_id, option_value):
    while True:
        try:
            selRANK = Select(browser.find_element_by_id(option_id))
            selRANK.select_by_value(option_value)
            break
        except StaleElementReferenceException:
            selRANK = Select(browser.find_element_by_id(option_id))
            selRANK.select_by_value(option_value)

# 定義增加新篩選條件方程式
def add():
    global num_filter
    global filter_choices
    filter_label = tk.Label(window, text = '第{}個篩選方法:'.format(num_filter + 1), justify = tk.RIGHT, width = 50)
    filter_label.place(x = 10, y = 70 + 30*num_filter, width = 100, height = 20)
    globals()['choice' + str(num_filter)] = tt.Combobox(window, width = 50, values = filter_choices)
    globals()['choice' + str(num_filter)].place(x = 110, y = 70 + 30*num_filter, width = 100, height = 20)
    globals()['tend_choice' + str(num_filter)] = tt.Combobox(window, width = 50, values = tendency)
    globals()['tend_choice' + str(num_filter)].place(x = 240, y = 70 + 30*num_filter, width = 100, height = 20)
    num_filter += 1
    if num_filter <= (len(filter_choices) - 1):
        add_filter = tk.Button(window, text = '增加新篩選方法', width = 40, command = add)
        add_filter.place(x = 60, y = 70 + 30*(num_filter), width = 100, height = 20)

# 定義解說方程式
def information():
    show = ''
    for j in range(len(inf)):
        show += inf[j][0] + ': ' + inf[j][1] + '\n\n'

    tk.messagebox.showinfo('提示', show) 
    
# 定義加入投資組合方程式
def add_portfolio(company_data):
    global portfolio
    global show_portfolio
    portfolio.append(company_data)
    show_portfolio.append(company_data[2])
    consider_company = tk.Label(window, text = show_portfolio)
    consider_company.place(x = 800, y = 50, width = 400, height = 20)
    
# 定義取消方程式
def c():
    global portfolio
    global show_portfolio
    global portfolio_category
    global portfolio_tick
    global portfolio_name
    global portfolio_price
    global portfolio_volumn
    
    portfolio = []
    show_portfolio = []
    consider_company = tk.Label(window, text = show_portfolio)
    consider_company.place(x = 800, y = 50, width = 400, height = 20)
    portfolio_category.delete(0, tk.END)
    portfolio_tick.delete(0, tk.END)
    portfolio_name.delete(0, tk.END)
    portfolio_price.delete(0, tk.END)
    portfolio_volumn.delete(0, tk.END)
    
# 定義計算投資組合的方程式
def okok():
    global portfolio
    global show_portfolio
    global portfolio_category
    global portfolio_tick
    global portfolio_name
    global portfolio_price
    global portfolio_volumn
    
    portfolio_category.delete(0, tk.END)
    portfolio_tick.delete(0, tk.END)
    portfolio_name.delete(0, tk.END)
    portfolio_price.delete(0, tk.END)
    portfolio_volumn.delete(0, tk.END)
    
    p = list(set(show_portfolio))
    
    if len(p) != len(show_portfolio):
        portfolio_category.insert(0, '有公司重複')
        portfolio_category.insert(1, '請重新選擇')
    elif len(portfolio) == 0:
        portfolio_category.insert(0, '無公司資料')
    else:
        for i in range(len(portfolio)):
            portfolio_category.insert(i, portfolio[i][0])
            portfolio_tick.insert(i, portfolio[i][1])
            portfolio_name.insert(i, portfolio[i][2])
            portfolio_price.insert(i, portfolio[i][3])
            portfolio_volumn.insert(i, portfolio[i][4])
        
        pr = ['產業', '代號', '公司名稱', '股價', '成交量']
        portfolio_category.insert(0, pr[0])
        portfolio_tick.insert(0, pr[1])
        portfolio_name.insert(0, pr[2])
        portfolio_price.insert(0, pr[3])
        portfolio_volumn.insert(0, pr[4])
            
            
       
        
# 定義加入篩選條件方程式
def check_choice():
    global num_filter
    global filter_condition
    global tendency_condition
    global company
    filter_condition = []
    tendency_condition = []
    company = []
    
    for i in range((num_filter)):
        if globals()['choice' + str(i)].get() == "風險":
            filter_condition.append('一年(β)')
        elif globals()['choice' + str(i)].get() == "成交量":
            filter_condition.append('成交張數')
        elif globals()['choice' + str(i)].get() == "股價":
            filter_condition.append('成交')
        elif globals()['choice' + str(i)].get() == "營收":
            filter_condition.append('營收(億)')
            
        if globals()['tend_choice' + str(i)].get() == '高':
            tendency_condition.append('高')
        elif globals()['tend_choice' + str(i)].get() == '中':
            tendency_condition.append('中')
        elif globals()['tend_choice' + str(i)].get() == '低':
            tendency_condition.append('低')
                    
# 定義篩選方程式
def start_sort():
    global num_filter
    global filter_condition
    global tendency_condition

    # 今天時間
    now = datetime.now()
    month = str(now.month).zfill(2)
    day = str(now.day).zfill(2)
    today_date = '{}{}{}'.format(now.year, month, day)

    df = pd.read_csv('20201228.csv')#.format(today_date))
    for i in range(len(df.index)):
        if pd.isnull(df.loc[i]['產業類別']):
            df = df.drop([i])
    df = df.replace('No Data', np.nan)
    df = df.replace(',','', regex=True)
    df["成交張數"] = pd.to_numeric(df["成交張數"], downcast="integer")
    df["成交"] = pd.to_numeric(df["成交"], downcast="float")
    df["一年(β)"] = pd.to_numeric(df["一年(β)"], downcast="float")
    df["營收(億)"] = pd.to_numeric(df["營收(億)"], downcast="float")
    
    nothing = tk.Label(window, justify = tk.RIGHT, width = 50)
    nothing.place(x = -100, y = 270, width = 400, height = 860)
    nothing = tk.Label(window, justify = tk.RIGHT, width = 50)
    nothing.place(x = 400, y = 0, width = 400, height = 860)
    if len(filter_condition) < 3:
        wrong = tk.Label(window, text = '請至少輸入3個篩選方法', justify = tk.RIGHT, width = 50)
        wrong.place(x = 30, y = 270, width = 150, height = 20) 
    else:
        for i in range(num_filter):
            df = df.sort_values(filter_condition[i])
            amount = len(df.index)
            if tendency_condition[i] == '低':
                max_amount = round(amount*0.3)
                df = df[:max_amount]
            elif tendency_condition[i] == '中':
                min_amount = round(amount*0.3)
                max_amount = round(amount*0.7)
                df = df[min_amount:max_amount]
            elif tendency_condition[i] == '高':
                min_amount = round(amount*0.7)
                df = df[min_amount:]

        for i in range(len(df.index)):
            if i < 15:
                category = tk.Label(window, text = df.iloc[i]['產業類別'], justify = tk.RIGHT, width = 50)
                category.place(x = 10, y = 270 + 20*i, width = 150, height = 20)
                rank = tk.Label(window, text = str(i+1), justify = tk.RIGHT, width = 50)
                rank.place(x = 10, y = 270 + 20*i, width = 20, height = 20)
                tick = tk.Label(window, text = df.iloc[i]['代號'], justify = tk.RIGHT, width = 50)
                tick.place(x = 140, y = 270 + 20*i, width = 30, height = 20)
                company_name1 = tk.Button(window, text = df.iloc[i]['名稱'], justify = tk.RIGHT, width = 50, command = lambda i = i: add_portfolio(company[i]))
                company_name1.place(x = 190, y = 270 + 20*i, width = 70, height = 20)
                price = tk.Label(window, text = df.iloc[i]['成交'], justify = tk.RIGHT, width = 50)
                price.place(x = 275, y = 270 + 20*i, width = 40, height = 20)
                volumn = tk.Label(window, text = int(df.iloc[i]['成交張數']), justify = tk.RIGHT, width = 50)
                volumn.place(x = 335, y = 270 + 20*i, width = 60, height = 20)
                company.append([category.cget('text'), tick.cget('text'), company_name1.cget('text'), price.cget('text'), volumn.cget('text')])

            elif i >= 15 and i < 41:
                rank = tk.Label(window, text = '排名')
                rank.place(x = 405, y = 30, width = 30, height = 20)
                category = tk.Label(window, text = '產業')
                category.place(x = 470, y = 30, width = 30, height = 20)
                tick = tk.Label(window, text = '股票代號')
                tick.place(x = 530, y = 30, width = 50, height = 20)
                company_name = tk.Label(window, text = '公司名稱')
                company_name.place(x = 600, y = 30, width = 50, height = 20)
                price = tk.Label(window, text = '股價')
                price.place(x = 680, y = 30, width = 30, height = 20)
                volumn = tk.Label(window, text = '成交量')
                volumn.place(x = 745, y = 30, width = 40, height = 20)
                
                category = tk.Label(window, text = df.iloc[i]['產業類別'], justify = tk.RIGHT, width = 50)
                category.place(x = 410, y = 50 + 20*(i-15), width = 150, height = 20)
                rank = tk.Label(window, text = str(i+1), justify = tk.RIGHT, width = 50)
                rank.place(x = 410, y = 50 + 20*(i-15), width = 20, height = 20)
                tick = tk.Label(window, text = df.iloc[i]['代號'], justify = tk.RIGHT, width = 50)
                tick.place(x = 540, y = 50 + 20*(i-15), width = 30, height = 20)
                company_name1 = tk.Button(window, text = df.iloc[i]['名稱'], justify = tk.RIGHT, width = 50, command = lambda i = i: add_portfolio(company[i]))
                company_name1.place(x = 590, y = 50 + 20*(i-15), width = 70, height = 20)
                price = tk.Label(window, text = df.iloc[i]['成交'], justify = tk.RIGHT, width = 50)
                price.place(x = 675, y = 50 + 20*(i-15), width = 40, height = 20)
                volumn = tk.Label(window, text = int(df.iloc[i]['成交張數']), justify = tk.RIGHT, width = 50)
                volumn.place(x = 735, y = 50 + 20*(i-15), width = 60, height = 20)
                company.append([category.cget('text'), tick.cget('text'), company_name1.cget('text'), price.cget('text'), volumn.cget('text')])

            else:
                how_many = len(df.index) - 41
                too_many = tk.Label(window, text = '(尚餘{}筆資料未顯示...)'.format(how_many), justify = tk.RIGHT, width = 50)
                too_many.place(x = 420, y = 50 + 20*(i-15), width = 150, height = 20) 
                break   
                
                
# 主要程式
wd = os.getcwd()  # 目前檔案路徑

# 今天時間
now = datetime.now()
month = str(now.month).zfill(2)
day = str(now.day).zfill(2)
today_date = '{}{}{}'.format(now.year, month, day)

filter_choices = ['風險', '成交量', '股價', '營收']
tendency = ['高', '中', '低']
inf = [['風險', 'beta係數衡量資產系統性風險的指標，代表市場大盤報酬與資產報酬變動相關性。當beta愈高，代表當總體市場變化時，此資產隨之變動程度越高，及風險越高。'], ['成交量', '此資產過去一日之總成交張數。當股票成交量高且價格上漲時，是多頭之基本趨勢。代表買賣積極、交易熱絡之狀況。而若成交量過低，可能代表此資產流動性較差，須多加注意其換手風險。'], ['股價', '此資產最近之收盤價。'], ['營收', '此資產之上月營收。']]

# 判別檔案是否存在
file = '20201228.csv'#.format(today_date)
while True:
    if os.path.isfile(file):
        # 執行篩選器
        window = tk.Tk()
        window.title('股票推薦')
        window.geometry('1200x600')
        
        filter_condition = []
        tendency_condition = []
        portfolio = []
        company = []
        show_portfolio = []
        
        num_filter = 3
        statement1 = tk.Label(window, text = r'篩選條件(至少三個)')
        statement1.place(x = 85, y = 40, width = 150, height = 20)
        statement1 = tk.Label(window, text = '偏好程度')
        statement1.place(x = 245, y = 40, width = 100, height = 20)
        information_button = tk.Button(window, text = r'?', width = 10, command = information)
        information_button.place(x = 70, y = 40, width = 20, height = 20)
        c_c = tk.Label(window, text = '預計選入投資組合的公司(最多10個)')
        c_c.place(x = 800, y = 30, width = 400, height = 20)
        ok = tk.Button(window, text = '確認', command = okok)
        ok.place(x = 950, y = 70, width = 50, height = 20)
        cancel = tk.Button(window, text = '取消', command = c)
        cancel.place(x = 1000, y = 70, width = 50, height = 20)
        portfolio_show_str = tk.Label(window, text = '投資組合')
        portfolio_show_str.place(x = 950, y = 110, width = 100, height = 20)
        portfolio_category = tk.Listbox(window, width = 380)
        portfolio_category.place(x = 850, y = 130, width = 100, height = 180)
        portfolio_tick = tk.Listbox(window, width = 380)
        portfolio_tick.place(x = 950, y = 130, width = 50, height = 180)
        portfolio_name = tk.Listbox(window, width = 380)
        portfolio_name.place(x = 1000, y = 130, width = 75, height = 180)
        portfolio_price = tk.Listbox(window, width = 380)
        portfolio_price.place(x = 1075, y = 130, width = 50, height = 180)
        portfolio_volumn = tk.Listbox(window, width = 380)
        portfolio_volumn.place(x = 1125, y = 130, width = 50, height = 180)

        for i in range(3):
            filter_label = tk.Label(window, text = '第{}個篩選方法:'.format(i+1), justify = tk.RIGHT, width = 50)
            filter_label.place(x = 10, y = 70 + 30*i, width = 100, height = 20)
            globals()['choice' + str(i)] = tt.Combobox(window, width = 50, values = filter_choices)
            globals()['choice' + str(i)].place(x = 110, y = 70 + 30*i, width = 100, height = 20)
            globals()['tend_choice' + str(i)] = tt.Combobox(window, width = 50, values = tendency)
            globals()['tend_choice' + str(i)].place(x = 240, y = 70 + 30*i, width = 100, height = 20)


        
        add_filter = tk.Button(window, text = '增加新篩選方法', width = 40, command = add)
        add_filter.place(x = 60, y = 160, width = 100, height = 20)
        add_in = tk.Button(window, text = '加入篩選條件', width = 40, command = check_choice)
        add_in.place(x = 150, y = 200, width = 100, height = 20)
        start = tk.Button(window, text = '開始篩選', width = 40, command = start_sort)
        start.place(x = 270, y = 200, width = 100, height = 20)
        
        rank = tk.Label(window, text = '排名')
        rank.place(x = 5, y = 240, width = 30, height = 20)
        category = tk.Label(window, text = '產業')
        category.place(x = 70, y = 240, width = 30, height = 20)
        tick = tk.Label(window, text = '股票代號')
        tick.place(x = 130, y = 240, width = 50, height = 20)
        company_name = tk.Label(window, text = '公司名稱')
        company_name.place(x = 200, y = 240, width = 50, height = 20)
        price = tk.Label(window, text = '股價')
        price.place(x = 280, y = 240, width = 30, height = 20)
        volumn = tk.Label(window, text = '成交量')
        volumn.place(x = 345, y = 240, width = 40, height = 20)
        

        window.mainloop()
        
        break
    else:
        # 爬取資料
        
        # 指標名稱
        tick_name_1 = ['代號', '名稱', '股價日期', '成交', '漲跌價', '漲跌幅', '成交張數', '2日累計漲跌價', '2日累計漲跌幅', '3日累計漲跌價', '3日累計漲跌幅', '5日累計漲跌價', '5日累計漲跌幅', '10日累計漲跌價', '10日累計漲跌幅', '一個月累計漲跌價', '一個月累計漲跌幅']
        tick_name_2 = ['代號', '名稱', '市場', '股價日期', '成交', '漲跌價', '漲跌幅', '5日(β)', '10日(β)', '一個月(β)', '三個月(β)', '半年(β)', '一年(β)', '二年(β)', '三年(β)', '五年(β)', '十年(β)', '十五年(β)', '二十年(β)']
        tick_name_3 = ['代號', '名稱', '市場', '股價日期', '成交', '漲跌價', '漲跌幅', '成交張數', '營收月份', '單月營收(億)', '單月營收月增(億)', '單月營收月增(%)', '單月營收年增(億)', '單月營收年增(%)', '累月營收(億)', '累月營收年增(億)', '累月營收年增(%)', '月營收增減註記']
        tick_name_4 = ['代號', '名稱', '成交', '漲跌價', '漲跌幅', '資券日期', '融資買進', '融資賣出', '融資現償', '融資增減', '融資餘額', '融資使用(%)', '融資餘額佔發行量', '融券買進', '融券賣出', '融券現償', '融券增減', '融券餘額', '融券使用(%)', '融券餘額佔發行量', '資券互抵', '資券當沖(%)', '券資比(%)']
        tick_name_5 = ['代號', '名稱', '成交', '財報季度', '營收(億)', '營收成長(%)', '毛利(億)', '毛利成長(%)', '淨利(億)', '淨利成長(%)', '毛利(%)', '毛率增減', '淨利(%)', '淨率增減', 'EPS(元)', 'EPS增減(元)', '單季ROE(%)', '單季ROE增減', '年估ROE(%)', '單季ROA(%)', '單季ROA增減', '年估ROA(%)', '財報評分']
        tick_name_6 = ['代號', '名稱', '股價日期', '成交', '漲跌價', '漲跌幅', '股利發放年度', '最後成交價', '所屬EPS', '現金股利', '股票股利', '合計股利', '現金殖利率', '股票殖利率', '合計殖利率', '盈餘配息率', '盈餘配股率', '盈餘總分配率', '現金股利發放日']
        tick_total = [tick_name_1, tick_name_2, tick_name_3, tick_name_4, tick_name_5, tick_name_6]

        # 指標內容
        select = ['漲跌及成交統計', '風險指標', '營收狀況', '融資融券', '季獲利能力', '股利政策']  

        # 產業類別
        category = ['水泥工業', '食品工業', '塑膠工業', '紡織纖維', '電機機械', '電器電纜', '生技醫療業', '化學工業', '玻璃陶瓷', '造紙工業', '鋼鐵工業',  '橡膠工業', '汽車工業', '電腦及週邊設備業', '半導體業', '電子零組件業', '其他電子業', '通信網路業', '資訊服務業', '建材營造業', '航運業', '觀光事業', '銀行業', '保險業', '金控業', '貿易百貨業', '光電業', '電子通路業', '證券業', '其他業', '油電燃氣業', '電子商務', '文化創意業', '農業科技業', '存託憑證']


        # 今天時間
        now = datetime.now()
        month = str(now.month).zfill(2)
        day = str(now.day).zfill(2)
        today_date = '{}{}{}'.format(now.year, month, day)

        # 爬蟲以存取檔案
        with open('{}.csv'.format(today_date), 'w+', newline = '', encoding='utf-8-sig') as csvFile:
            writer = csv.writer(csvFile)

            for l in range(len(category)):
                industry = urllib.parse.quote(category[l])
                tick_name = []
                stock_data = []

                # 選擇資料項目
                for i in range(len(select)):
                    while True:
                        try:
                            # 開啟網頁
                            browser = webdriver.Chrome()
                            url = 'https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT=%E5%85%A8%E9%83%A8&INDUSTRY_CAT={}'.format(industry)
                            browser.get(url)
                            browser.implicitly_wait(60)
                            time.sleep(3)

                            # 選擇指標
                            choose('selSHEET', select[i])
                            browser.implicitly_wait(60)
                            time.sleep(7)
                            tick = tick_total[i]
                            data = browser.find_element_by_id('tblStockList')
                            data = data.text.split('\n')
                            data_total = []

                            if i == 0:
                                for d in data:
                                    d = d.split()
                                    if len(d) == 1:
                                        continue
                                    else:
                                        stock_data.append([])

                            for d in data:
                                d = d.split()
                                if len(d) == 1:
                                    continue
                                else:
                                    if i == 0 or i == 1:
                                        diff = len(tick) - len(d)
                                        for j in range(diff):
                                            d.append('No Data')
                                    if i == 2:
                                        diff = len(tick) - len(d)
                                        for j in range(diff):
                                            d.insert(-2, 'No Data') 
                                    if i == 3:
                                        if len(d) == 22:
                                            for m in [22]:
                                                d.insert(m, 'No Data')
                                        elif len(d) == 21:
                                            for m in [11, 18]:
                                                d.insert(m, 'No Data')
                                        elif len(d) == 20:
                                            for m in [11, 18, 21]:
                                                d.insert(m, 'No Data')
                                        elif len(d) == 16:
                                            for m in [9, 11, 16, 18, 20, 21, 22]:
                                                d.insert(m, 'No Data')
                                        elif len(d) == 14:
                                            for m in [9, 11, 12, 16, 18, 19, 20, 21, 22]:
                                                d.insert(m, 'No Data')
                                    if i == 4:
                                        if len(d) == 19:
                                            for m in [6, 7, 10, 11]:
                                                d.insert(m, 'No Data')
                                        else:
                                            diff = len(tick) - len(d)
                                            for j in range(diff):
                                                d.append('No Data')
                                    if i == 5:
                                        if len(d) == 15:
                                            for m in [8, 15, 16, 17]:
                                                d.insert(m, 'No Data')
                                        if len(d) == (len(tick)-1):
                                            d.append('No Data')
                                        else:
                                            diff = len(tick) - len(d)
                                            for j in range(diff):
                                                d.append('No Data')
                                    data_total.append(d)

                            for j in range(len(tick)):
                                if tick[j] in tick_name:
                                    continue
                                else:
                                    tick_name.append(tick[j])
                                    for k in range(len(data_total)): 
                                        stock_data[k].append(data_total[k][j])
                            browser.close()
                            break
                        except:
                            print('{} 的 {} 資料錯誤，正重新抓取'.format(category[l], select[i]))
                            browser.close()
                            time.sleep(4)
                            continue

                if l == 0:
                    writer.writerow(['產業類別'] + tick_name)
                for i in range(len(stock_data)):
                     writer.writerow([category[l]] + stock_data[i])

                print('{} 資料抓取完畢'.format(category[l]))

        csvFile.close()
        continue


In [64]:
a = []
for i in range(5):
    a.insert(i, i)
print(a)

[0, 1, 2, 3, 4]
